# Взаимодействие с гигачатом
В этом гугл-коллабе вы можете провзаимодействовать с гигачатом: задать запрос или провести диалог, с заданными промтом. Рекомендуется выполнять все ячейки последовательно, иначе может не установиться определенная библиотека.

In [1]:
pip install gigachat #Устанавливаем библиотеку Gigachat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 3.5 MB/s eta 0:00:00


In [2]:
import requests #Данная библиотека (requests) позволяет взаимодействовать с ссылками и сайтами. Нам она понадобиться чтобы задавать запрос гигачату
from gigachat import GigaChat
!curl -k "https://gu-st.ru/content/lending/russian_trusted_root_ca_pem.crt" -w "\n" >> $(python -m certifi) #Для взаимодействия с нашей нейронкой сбер требует так называемые сертификаты Минцифры, эта строчка, грубо говоря, позволяет обойти это ограничение

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2088  100  2088    0     0   2659      0 --:--:-- --:--:-- --:--:--  2659


# Аутентификационный ключ
С помощью этого ключа, мы как-бы 'входим в аккаунт' чтобы обращаться к нейронке. В данном случае вы входите в мой аккаунт. У каждого аккаунта ограниченное кол-во токенов (запросов). В принципе токенов очень много, поэтому можно не волноваться о том, что они закончаться. Если они  и закончаться, то будем использовать чей нибудь другой аутентификационный ключ, для этого надо просто вместо моего ключа, вписать свой (йоу).

In [3]:
auth_key = "MDE5OTc4MTEtM2NjMS03ODNkLTkxYzAtMmM4MzZhN2UxNzM2OmE0YmY3NTdkLTkxNjItNGRjNi04ZDA1LTBiOTM4ZTRjM2JjOA=="

In [ ]:
#Данная ячейка позваоляет задать запрос к нашей нейронке
from gigachat import GigaChat
import re
giga = GigaChat(
    credentials=auth_key,
    model="GigaChat-2-Max",  # укажите нужную модель
    scope="GIGACHAT_API_PERS"  # укажите область видимости
)
response = giga.chat("Расскажи про имя Илья") #Здесь вы можете вписать свой запрос в кавычках
answer = response.choices[0].message.content
clean_answer = re.sub(r'[*#]+', '', answer)
print(clean_answer)

Имя Илья имеет древнееврейское происхождение (עלייה — Эли́яhу), переводится оно как «мой Бог есть Яхве», либо просто «Бог мой». В русском языке произносится чаще всего именно как Илья, хотя существует и другая форма имени — Элий/Элиан.

На Руси имя стало широко известно благодаря святому пророку Илье, почитаемому ещё с ветхозаветной эпохи: этот библейский персонаж считается одним из величайших святых и чудотворцев Ветхого Завета. Русская православная церковь особо чтит день памяти святого пророка Ильи — 2 августа (по новому стилю). Этот праздник традиционно сопровождается народными обрядами и поверьями («Ильин день»), связанными с грозой, молнией и дождём, поскольку святой Илия считался покровителем небесных стихий.

Особенности характера людей с именем Илья часто связывают с такими чертами, как справедливость, решительность, доброта и смелость. Это человек открытый, добрый сердцем, способный проявлять инициативу и защищать слабых. Имя обладает положительной энергетикой, что помогает л

#Лангчейн
Итак, вроде мы уже +- понимаем как задать запрос гигачату, но эти запросы, но когда мы делаем так, как в ячейке выше, нейронка не запоминает запросы, поэтому каждый новый запрос для нее первый. Это не позволяет нам нормально создать диалог с ИИ. Конечно есть способ как это исправить - лангчейн. Грубо говоря, это технология позволяет нейронке запоминать контектс предыдущих запрос, из-за чего можно вытстроить нормальный диалог


In [5]:
pip install -U langchain-gigachat #Устанавливаем необхожимую библиотеку

In [7]:
#В данной ячейке реализуется один запрос, но уже используя библиотеку лангчейна.
from langchain_gigachat.chat_models import GigaChat

giga = GigaChat(
    # Для авторизации запросов используйте ключ, полученный в проекте GigaChat API
    credentials="MDE5OTc4MTEtM2NjMS03ODNkLTkxYzAtMmM4MzZhN2UxNzM2OmE0YmY3NTdkLTkxNjItNGRjNi04ZDA1LTBiOTM4ZTRjM2JjOA==",
    verify_ssl_certs=False,
)
response = giga.invoke("Кто такой Декстер?")


if hasattr(response, 'content'):
    final_answer = response.content.strip()
elif isinstance(response, str):
    final_answer = response.strip()  # Обрезаем лишние пробелы
else:
    raise ValueError(f"Невозможно обработать ответ: {type(response)}")

# Выводим только чистое сообщение
print(final_answer)



Не помню конкретного твоего вопроса про Декстера ранее (у меня нет памяти о предыдущих диалогах между пользователями). Можешь напомнить тему, чтобы я мог уточнить детали?


In [9]:
#Вот в этой ячейке вы можете воссоздать полноценный диалог
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain_gigachat.chat_models import GigaChat

# Настройка подключения к GigaChat
giga = GigaChat(model="GigaChat-2-Max", credentials="MDE5OTc4MTEtM2NjMS03ODNkLTkxYzAtMmM4MzZhN2UxNzM2OmE0YmY3NTdkLTkxNjItNGRjNi04ZDA1LTBiOTM4ZTRjM2JjOA==")

# Создание компонента памяти
memory = ConversationBufferMemory(return_messages=True)

# Определение шаблона промптинга
prompt_template = """Ты бот, который создает планы обучения для людей. Учитывай контекст предыдущих сообщений. При запросе пользователя плана обучения, учитывай требования к плану обучения от пользователя. План должен выглядеть в виде таблицы.
{history}
Human: {input}
Assistant:"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=prompt_template)

# Создание цепи с компонентом памяти
conversation_chain = ConversationChain(llm=giga, verbose=False, memory=memory, prompt=PROMPT)

def chat(prompt):
    result = conversation_chain.predict(input=prompt)
    return result.strip()

# Пример использования
while True:
    user_input = input("Вы: ")
    if user_input.lower() == "выйти":
        break
    answer = chat(user_input)
    print("Ассистент:", answer)

Вы: Привет!
Ассистент: Привет! Для какого направления тебе нужен план обучения? Уточни цель, уровень подготовки и желаемые сроки освоения материала.
Вы: Ого, ты такой умный и забавный, ахахаха. Чтож, я бы хотел выучить java
Ассистент: | Этап | Тема                     | Содержание                          |
|------|--------------------------|-------------------------------------|
| 1    | Основы Java              | Установка среды разработки <br> Переменные, типы данных, операторы |
| 2    | Управление потоком       | Условия (if-else), циклы             |
| 3    | Объектно-ориентированное программирование | Классы, объекты, методы, наследование, полиморфизм |
| 4    | Работа с коллекциями     | Массивы, списки, множества           |
| 5    | Исключения               | Обработка ошибок                    |
| 6    | Потоки ввода-вывода      | Файлы, потоки                       |
| 7    | Многопоточность         | Создание потоков, синхронизация   |
| 8    | Тестирование             | U